In [2]:
"""
LINCS 2020 Chemical Filters
根据DrugReflector论文SI第2页实现化学过滤
"""

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
import os
warnings.filterwarnings('ignore')

# RDKit imports
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, AllChem, FilterCatalog
    from rdkit.Chem.FilterCatalog import FilterCatalogParams
    RDKIT_AVAILABLE = True
except ImportError:
    print("⚠️  RDKit not installed. Install with: conda install -c conda-forge rdkit")
    RDKIT_AVAILABLE = False


class ChemicalFilter:
    """
    化学过滤器 - 根据DrugReflector SI实现
    
    Filters applied (SI page 2):
    1. Molecular weight: 60-1000 Da (inclusive)
    2. No more than 1 covalent motif (SMARTS-defined)
    3. No more than 9 NIBR structure flags
    4. Pass BRENK criteria
    5. Must not match 30 SMARTS patterns (not disclosed)
    """
    
    def __init__(self, compound_info_path: str, nibr_filter_csv: Optional[str] = None):
        """
        初始化化学过滤器
        
        参数：
            compound_info_path: compoundinfo_beta.txt路径
            nibr_filter_csv: NIBR过滤器CSV文件路径（可选）
        """
        if not RDKIT_AVAILABLE:
            raise ImportError("RDKit is required for chemical filtering")
        
        self.compound_info_path = Path(compound_info_path)
        self.compound_info = None
        self.filtered_compounds = set()
        
        # 初始化BRENK过滤器
        self.brenk_catalog = self._init_brenk_filter()
        
        # 初始化共价基团SMARTS
        self.covalent_smarts = self._init_covalent_smarts()
        
        # 🔥 新增：初始化NIBR过滤器目录
        self.nibr_catalog = self._init_nibr_filter(nibr_filter_csv)
        
        print(f"{'='*80}")
        print(f"🧪 Chemical Filter Initialized")
        print(f"{'='*80}")
        print(f"Based on DrugReflector SI (page 2)")
        print(f"{'='*80}\n")
    
    def _init_brenk_filter(self):
        """初始化BRENK过滤器"""
        params = FilterCatalogParams()
        params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
        return FilterCatalog.FilterCatalog(params)
    
    def _init_covalent_smarts(self) -> List[str]:
        """
        共价基团SMARTS模式
        
        参考：
        - Backman et al. 2019 (ChEMBL structural alerts)
        - Common covalent warheads in drug discovery
        """
        return [
            # Michael acceptors
            'C=CC(=O)',  # α,β-unsaturated carbonyl
            'C=CC(=O)N',  # acrylamide
            'C=CC#N',  # acrylonitrile
            
            # Electrophilic carbonyls
            '[C;!R](=O)Cl',  # acyl chloride
            '[C;!R](=O)O[C;!R](=O)',  # anhydride
            'C(=O)N=[N+]=[N-]',  # acyl azide
            
            # Epoxides and aziridines
            'C1OC1',  # epoxide
            'C1NC1',  # aziridine
            
            # Haloacetamides
            'ClCC(=O)N',  # chloroacetamide
            'BrCC(=O)N',  # bromoacetamide
            
            # Aldehydes (reactive)
            '[CH;!R]=O',  # aldehyde
            
            # Isocyanates/isothiocyanates
            'N=C=O',  # isocyanate
            'N=C=S',  # isothiocyanate
            
            # Sulfonyl halides
            'S(=O)(=O)Cl',  # sulfonyl chloride
            'S(=O)(=O)F',  # sulfonyl fluoride
            
            # Nitriles (activated)
            '[C;!R]#N',  # nitrile (non-aromatic)
            
            # Peroxides
            'OO',  # peroxide
            
            # Beta-lactams (strained)
            'C1(=O)NCC1',  # beta-lactam
            
            # Activated esters
            'C(=O)OC(=O)',  # activated ester
            
            # Quinones
            'C1=CC(=O)C=CC1=O',  # quinone
        ]
    
    def _init_nibr_filter(self, csv_path: Optional[str] = None):
        """
        初始化NIBR过滤器目录（基于官方444个SMARTS）
        
        参数：
            csv_path: NIBR过滤器CSV文件路径（可选）
            
        返回：
            FilterCatalog对象
        """
        # 🔥 确定CSV文件路径
        if csv_path is None:
            # 默认路径：当前脚本目录下的chem_filter文件夹
            try:
                # 尝试使用 __file__（在脚本模式下）
                script_dir = Path(__file__).parent
            except NameError:
                # 在交互式环境中使用当前工作目录
                script_dir = Path(os.getcwd())
            csv_path = script_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
        else:
            csv_path = Path(csv_path)
        
        if not csv_path.exists():
            print(f"⚠️  NIBR filter CSV not found: {csv_path}")
            print(f"   Using fallback: empty NIBR catalog")
            return FilterCatalog.FilterCatalog()
        
        print(f"📖 Loading NIBR filters from: {csv_path.name}")
        
        # 读取CSV并构建FilterCatalog
        try:
            nibr_df = pd.read_csv(csv_path)
            print(f"   ✓ Loaded {len(nibr_df)} NIBR filter patterns")
            
            nibr_catalog = FilterCatalog.FilterCatalog()
            
            n_added = 0
            for idx, row in nibr_df.iterrows():
                try:
                    # 提取参数
                    pattern_name = row['PATTERN_NAME']
                    smarts = row['SMARTS']
                    min_count = 1 if row['MIN_COUNT'] == 0 else int(row['MIN_COUNT'])
                    severity = int(row['SEVERITY_SCORE'])
                    covalent = int(row['COVALENT'])
                    special_mol = int(row['SPECIAL_MOL'])
                    set_name = row['SET_NAME']
                    
                    # 构建过滤器名称（与官方代码一致）
                    filter_name = f"{pattern_name}_min({min_count})__{severity}__{covalent}__{special_mol}"
                    
                    # 创建SMARTS匹配器
                    matcher = FilterCatalog.SmartsMatcher(filter_name, smarts, min_count)
                    
                    # 添加到目录
                    entry = FilterCatalog.FilterCatalogEntry(filter_name, matcher)
                    entry.SetProp('Scope', set_name)
                    entry.SetProp('Severity', str(severity))
                    nibr_catalog.AddEntry(entry)
                    
                    n_added += 1
                    
                except Exception as e:
                    print(f"   ⚠️  Failed to add filter {idx}: {e}")
                    continue
            
            print(f"   ✓ Successfully added {n_added} NIBR filters to catalog")
            return nibr_catalog
            
        except Exception as e:
            print(f"   ❌ Error loading NIBR filters: {e}")
            print(f"   Using fallback: empty NIBR catalog")
            return FilterCatalog.FilterCatalog()
    
    def load_compound_info(self):
        """加载化合物信息"""
        print(f"📖 Loading compound information...")
        
        if not self.compound_info_path.exists():
            raise FileNotFoundError(
                f"Compound info file not found: {self.compound_info_path}"
            )
        
        self.compound_info = pd.read_csv(self.compound_info_path, sep='\t')
        
        print(f"   ✓ Loaded {len(self.compound_info):,} compounds")
        print(f"   ✓ Columns: {list(self.compound_info.columns)}")
        
        # 检查SMILES列
        smiles_col = None
        for col in ['canonical_smiles', 'smiles', 'SMILES']:
            if col in self.compound_info.columns:
                smiles_col = col
                break
        
        if smiles_col is None:
            raise ValueError(
                f"SMILES column not found. Available columns: {list(self.compound_info.columns)}"
            )
        
        self.smiles_col = smiles_col
        print(f"   ✓ Using '{smiles_col}' for molecular structures")
        
        # 检查有效SMILES数量
        valid_smiles = self.compound_info[smiles_col].notna().sum()
        print(f"   ✓ Compounds with valid SMILES: {valid_smiles:,}")
        
        return self.compound_info
    
    def filter_1_molecular_weight(self, min_mw=60, max_mw=1000) -> set:
        """
        Filter 1: 分子量过滤 (60-1000 Da)
        
        参数：
            min_mw: 最小分子量（默认60）
            max_mw: 最大分子量（默认1000）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 1: Molecular Weight ({min_mw}-{max_mw} Da)")
        print(f"{'='*80}")
        
        passed = set()
        failed_mw = []
        no_smiles = 0
        invalid_smiles = 0
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            # 检查SMILES是否存在
            if pd.isna(smiles) or smiles == '':
                no_smiles += 1
                continue
            
            # 解析分子
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                invalid_smiles += 1
                continue
            
            # 计算分子量
            mw = Descriptors.MolWt(mol)
            
            if min_mw <= mw <= max_mw:
                passed.add(pert_id)
            else:
                failed_mw.append((pert_id, mw))
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (out of MW range): {len(failed_mw):,} compounds")
        print(f"  ⚠️  No SMILES: {no_smiles:,} compounds")
        print(f"  ⚠️  Invalid SMILES: {invalid_smiles:,} compounds")
        
        if failed_mw:
            failed_df = pd.DataFrame(failed_mw, columns=['pert_id', 'MW'])
            print(f"\n  MW distribution of failed compounds:")
            print(f"    • Mean: {failed_df['MW'].mean():.1f} Da")
            print(f"    • Min: {failed_df['MW'].min():.1f} Da")
            print(f"    • Max: {failed_df['MW'].max():.1f} Da")
        
        return passed
    
    def filter_2_covalent_motifs(self, max_motifs=1) -> set:
        """
        Filter 2: 共价基团过滤 (≤1个)
        
        参数：
            max_motifs: 允许的最大共价基团数（默认1）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 2: Covalent Motifs (≤{max_motifs})")
        print(f"{'='*80}")
        print(f"Checking {len(self.covalent_smarts)} covalent SMARTS patterns...")
        
        passed = set()
        failed_counts = []
        
        # 预编译SMARTS
        smarts_mols = []
        for smarts in self.covalent_smarts:
            try:
                smarts_mol = Chem.MolFromSmarts(smarts)
                if smarts_mol:
                    smarts_mols.append(smarts_mol)
            except:
                print(f"  ⚠️  Invalid SMARTS: {smarts}")
        
        print(f"  ✓ Successfully compiled {len(smarts_mols)} SMARTS patterns")
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 计数匹配的共价基团
            motif_count = 0
            for smarts_mol in smarts_mols:
                if mol.HasSubstructMatch(smarts_mol):
                    motif_count += 1
            
            if motif_count <= max_motifs:
                passed.add(pert_id)
            else:
                failed_counts.append(motif_count)
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (>{max_motifs} motifs): {len(failed_counts):,} compounds")
        
        if failed_counts:
            print(f"\n  Distribution of failed compounds:")
            print(f"    • Mean motifs: {np.mean(failed_counts):.1f}")
            print(f"    • Max motifs: {max(failed_counts)}")
        
        return passed
    
    def filter_3_nibr_flags(self, max_flags=9) -> set:
        """
        Filter 3: NIBR结构标记 (≤9个)
        
        使用官方NIBR 444个SMARTS过滤器
        参考：Schuffenhauer et al. 2020, J. Med. Chem.
        
        参数：
            max_flags: 允许的最大标记数（默认9，符合SI要求）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 3: NIBR Structure Flags (≤{max_flags})")
        print(f"{'='*80}")
        print(f"Using official NIBR filter catalog (Schuffenhauer et al. 2020)")
        
        if self.nibr_catalog is None or self.nibr_catalog.GetNumEntries() == 0:
            print(f"  ⚠️  NIBR catalog is empty, skipping this filter")
            # 返回所有化合物（相当于不过滤）
            return set(self.compound_info['pert_id'].values)
        
        print(f"  ✓ Catalog contains {self.nibr_catalog.GetNumEntries()} filter patterns")
        
        passed = set()
        failed_counts = []
        flag_distribution = []
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 🔥 使用FilterCatalog获取所有匹配
            matches = self.nibr_catalog.GetMatches(mol)
            
            if len(matches) == 0:
                # 没有匹配任何flag，通过
                flag_count = 0
                passed.add(pert_id)
            else:
                # 🔥 计算flag数量（参考官方代码逻辑）
                # 只统计severity=1的FLAG，severity=2的EXCLUDE单独处理
                severity_scores = []
                for entry in matches:
                    # 提取severity信息
                    desc = entry.GetDescription()
                    # 格式：pattern_name_min(X)__severity__covalent__special_mol
                    parts = desc.split('__')
                    if len(parts) >= 2:
                        severity = int(parts[1])
                        severity_scores.append(severity)
                
                # 🔥 关键逻辑（与官方代码一致）：
                # 如果有severity=2（EXCLUDE），直接标记为10（必须排除）
                if 2 in severity_scores:
                    flag_count = 10  # 超过阈值，必须排除
                else:
                    # 否则，flag_count = severity=1的数量
                    flag_count = sum(1 for s in severity_scores if s == 1)
                
                flag_distribution.append(flag_count)
                
                if flag_count <= max_flags:
                    passed.add(pert_id)
                else:
                    failed_counts.append(flag_count)
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (>{max_flags} flags): {len(failed_counts):,} compounds")
        
        if flag_distribution:
            print(f"\n  Flag distribution (all compounds):")
            print(f"    • Mean flags: {np.mean(flag_distribution):.2f}")
            print(f"    • Median flags: {np.median(flag_distribution):.0f}")
            print(f"    • Max flags: {max(flag_distribution)}")
            
            # 统计各flag数量的分布
            from collections import Counter
            flag_counts = Counter(flag_distribution)
            print(f"\n  Flag count histogram:")
            for count in sorted(flag_counts.keys())[:15]:  # 只显示前15个
                n_compounds = flag_counts[count]
                print(f"    • {count} flags: {n_compounds:,} compounds")
        
        if failed_counts:
            print(f"\n  Failed compounds statistics:")
            print(f"    • Mean flags: {np.mean(failed_counts):.2f}")
            print(f"    • Max flags: {max(failed_counts)}")
        
        return passed
    
    def filter_4_brenk(self) -> set:
        """
        Filter 4: BRENK过滤器
        
        BRENK过滤器识别不良的化学基团和反应性官能团
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 4: BRENK Criteria")
        print(f"{'='*80}")
        
        passed = set()
        failed = []
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 检查BRENK过滤器
            matches = self.brenk_catalog.GetMatches(mol)
            
            if len(matches) == 0:
                passed.add(pert_id)
            else:
                failed.append((pert_id, len(matches)))
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (BRENK violations): {len(failed):,} compounds")
        
        return passed
    
    def apply_all_filters(self, target_compounds: Optional[set] = None) -> set:
        """
        应用所有化学过滤器
        
        返回：
            target_compounds: 需要过滤的化合物集合（如果提供，则只过滤这些化合物）
        """
        print(f"\n{'='*80}")
        print(f"🧪 APPLYING ALL CHEMICAL FILTERS")
        print(f"{'='*80}")
        
        if self.compound_info is None:
            self.load_compound_info()
        
        #  如果指定了目标化合物，只过滤这些
        if target_compounds is not None:
            print(f"\n⚠️  Filtering only {len(target_compounds):,} compounds present in training data")
            # 临时保存原始数据
            original_compound_info = self.compound_info.copy()
            # 只保留目标化合物
            self.compound_info = self.compound_info[
                self.compound_info['pert_id'].isin(target_compounds)
            ]
            print(f"   Compounds to check: {len(self.compound_info):,}")
        
        initial_compounds = len(self.compound_info)
        print(f"Initial compounds: {initial_compounds:,}")
        
        # 应用各个过滤器
        passed_mw = self.filter_1_molecular_weight()
        passed_covalent = self.filter_2_covalent_motifs()
        passed_nibr = self.filter_3_nibr_flags()
        passed_brenk = self.filter_4_brenk()
        
        # 取交集
        passed_all = passed_mw & passed_covalent & passed_nibr & passed_brenk
        
        # 如果是针对目标化合物的过滤，恢复原始数据
        if target_compounds is not None:
            self.compound_info = original_compound_info
        
        # 统计
        print(f"\n{'='*80}")
        print(f"📊 CHEMICAL FILTER SUMMARY")
        print(f"{'='*80}")
        print(f"  Initial compounds: {initial_compounds:,}")
        print(f"  Passed Filter 1 (MW): {len(passed_mw):,}")
        print(f"  Passed Filter 2 (Covalent): {len(passed_covalent):,}")
        print(f"  Passed Filter 3 (NIBR): {len(passed_nibr):,}")
        print(f"  Passed Filter 4 (BRENK): {len(passed_brenk):,}")
        print(f"  {'─'*80}")
        print(f"  ✅ Passed ALL filters: {len(passed_all):,}")
        print(f"  ❌ Removed: {initial_compounds - len(passed_all):,}")
        print(f"  Retention rate: {len(passed_all)/initial_compounds*100:.1f}%")
        
        self.filtered_compounds = passed_all
        return passed_all


def apply_chemical_filters_to_training_data(
    training_data_path: str,
    compound_info_path: str,
    output_path: Optional[str] = None,
    nibr_filter_csv: Optional[str] = None
) -> Dict:
    """
    对训练数据应用化学过滤器
    
    参数：
        training_data_path: 训练数据pkl文件路径
        compound_info_path: compoundinfo_beta.txt路径
        output_path: 输出路径（可选）
        nibr_filter_csv: NIBR过滤器CSV路径（可选）
    
    返回：
        过滤后的训练数据字典
    """
    print(f"\n{'='*80}")
    print(f"🔬 APPLYING CHEMICAL FILTERS TO TRAINING DATA")
    print(f"{'='*80}\n")
    
    # 1. 加载训练数据
    print(f"📖 Loading training data...")
    with open(training_data_path, 'rb') as f:
        training_data = pickle.load(f)
    
    initial_samples = len(training_data['X'])
    initial_compounds = len(training_data['compound_names'])
    
    print(f"   ✓ Loaded: {initial_samples:,} samples, {initial_compounds:,} compounds")
    
    # 获取训练数据中的化合物
    metadata = training_data['sample_meta']
    training_compounds = set(metadata['pert_id'].unique())
    print(f"   ✓ Training data contains: {len(training_compounds):,} unique compounds")
    
    # 2. 应用化学过滤器（只针对训练数据中的化合物）
    filter_obj = ChemicalFilter(compound_info_path, nibr_filter_csv=nibr_filter_csv)  # 🔥 传递参数
    filter_obj.load_compound_info()
    valid_compounds = filter_obj.apply_all_filters(target_compounds=training_compounds)
    
    # 3. 过滤训练数据
    print(f"\n{'='*80}")
    print(f"📊 FILTERING TRAINING DATA")
    print(f"{'='*80}")
    
    # valid_compounds现在是training_compounds的子集
    print(f"  Compounds in training data: {len(training_compounds):,}")
    print(f"  Compounds passed filters: {len(valid_compounds):,}")
    print(f"  Compounds to remove: {len(training_compounds - valid_compounds):,}")
    
    valid_mask = metadata['pert_id'].isin(valid_compounds)
    
    n_removed_samples = (~valid_mask).sum()
    n_removed_compounds = len(training_compounds - valid_compounds)  # 从训练数据中移除的

    print(f"  Removing {n_removed_samples:,} samples from {n_removed_compounds:,} compounds...")
    
    # 创建新的训练数据
    filtered_data = {
        'X': training_data['X'][valid_mask],
        'y': None,  # 需要重新编码
        'folds': training_data['folds'][valid_mask],
        'sample_meta': metadata[valid_mask].reset_index(drop=True),
        'metadata': metadata[valid_mask].reset_index(drop=True),
        'gene_names': training_data['gene_names'],
        'compound_names': None,  # 需要更新
        'pert_to_idx': None  # 需要重新构建
    }
    
    # 重新构建化合物映射和标签
    new_compound_names = sorted(list(valid_compounds))
    new_pert_to_idx = {pert: idx for idx, pert in enumerate(new_compound_names)}
    new_labels = np.array([
        new_pert_to_idx[pert] 
        for pert in filtered_data['sample_meta']['pert_id']
    ], dtype=np.int32)
    
    filtered_data['compound_names'] = new_compound_names
    filtered_data['pert_to_idx'] = new_pert_to_idx
    filtered_data['y'] = new_labels
    
    # 统计
    final_samples = len(filtered_data['X'])
    final_compounds = len(new_compound_names)
    
    if 'cell_iname' in filtered_data['sample_meta'].columns:
        final_cells = filtered_data['sample_meta']['cell_iname'].nunique()
    else:
        final_cells = 'Unknown'
    
    print(f"\n{'='*80}")
    print(f"✅ FILTERING COMPLETE")
    print(f"{'='*80}")
    print(f"  Before chemical filters:")
    print(f"    • Samples: {initial_samples:,}")
    print(f"    • Compounds: {initial_compounds:,}")
    print(f"  After chemical filters:")
    print(f"    • Samples: {final_samples:,} ({final_samples/initial_samples*100:.1f}%)")
    print(f"    • Compounds: {final_compounds:,} ({final_compounds/initial_compounds*100:.1f}%)")
    print(f"    • Cell lines: {final_cells}")
    print(f"  Removed:")
    print(f"    • Samples: {n_removed_samples:,}")
    print(f"    • Compounds: {n_removed_compounds:,}")
    
    # 与论文对比
    paper_samples = 425242
    paper_compounds = 9597
    paper_cells = 52
    
    print(f"\n📊 Comparison with paper (SI page 2):")
    print(f"  Paper: {paper_samples:,} obs, {paper_compounds:,} compounds, {paper_cells} cells")
    print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cells")
    print(f"  Match rate:")
    print(f"    • Samples: {final_samples/paper_samples*100:.1f}%")
    print(f"    • Compounds: {final_compounds/paper_compounds*100:.1f}%")
    if isinstance(final_cells, int):
        print(f"    • Cell lines: {final_cells/paper_cells*100:.1f}%")
    
    # 4. 保存
    if output_path is None:
        input_path = Path(training_data_path)
        output_path = input_path.parent / f"{input_path.stem}_chemfiltered.pkl"
    else:
        output_path = Path(output_path)
    
    print(f"\n💾 Saving filtered data to: {output_path}")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'wb') as f:
        pickle.dump(filtered_data, f, protocol=4)
    
    file_size_mb = output_path.stat().st_size / (1024**2)
    print(f"   ✓ Saved successfully! ({file_size_mb:.1f} MB)")
    
    return filtered_data


def main():
    """主程序 - 应用化学过滤器"""
    
    # 配置路径
    data_dir = "E:/科研/Models/drugreflector/datasets/LINCS2020"
    processed_dir = "E:/科研/Models/drugreflector/processed_data"
    
    training_data_path = Path(processed_dir) / "training_data_lincs2020_optimized_1201.pkl"
    compound_info_path = Path(data_dir) / "compoundinfo_beta.txt"
    output_path = Path(processed_dir) / "training_data_lincs2020_chemfiltered_1201.pkl"
    
    # NIBR过滤器CSV路径
    # CSV文件在当前脚本同级的chem_filter文件夹下
    try:
        # 尝试使用 __file__（在脚本模式下）
        script_dir = Path(__file__).parent
    except NameError:
        # 在交互式环境中使用当前工作目录
        script_dir = Path(os.getcwd())
    nibr_csv_path = script_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
    
    # 检查文件
    if not training_data_path.exists():
        print(f"❌ Training data not found: {training_data_path}")
        print(f"   Please run data preprocessing first!")
        return
    
    if not compound_info_path.exists():
        print(f"❌ Compound info not found: {compound_info_path}")
        return
    
    # 检查NIBR CSV文件
    if not nibr_csv_path.exists():
        print(f"⚠️  NIBR filter CSV not found: {nibr_csv_path}")
        print(f"   Will use fallback NIBR filters (less accurate)")
        nibr_csv_path = None
        
    # 应用过滤器
    try:
        filtered_data = apply_chemical_filters_to_training_data(
            training_data_path=str(training_data_path),
            compound_info_path=str(compound_info_path),
            output_path=str(output_path)
        )
        
        print(f"\n{'='*80}")
        print(f"✅ CHEMICAL FILTERING COMPLETE!")
        print(f"{'='*80}")
        print(f"📁 Final output: {output_path}")
        print(f"🎯 Ready for training!")
        
        return filtered_data
        
    except Exception as e:
        print(f"\n{'='*80}")
        print(f"❌ ERROR")
        print(f"{'='*80}")
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    filtered_data = main()


🔬 APPLYING CHEMICAL FILTERS TO TRAINING DATA

📖 Loading training data...
   ✓ Loaded: 583,491 samples, 13,823 compounds
   ✓ Training data contains: 13,823 unique compounds
📖 Loading NIBR filters from: SubstructureFilter_HitTriaging_wPubChemExamples.csv
   ✓ Loaded 444 NIBR filter patterns
   ✓ Successfully added 444 NIBR filters to catalog
🧪 Chemical Filter Initialized
Based on DrugReflector SI (page 2)

📖 Loading compound information...
   ✓ Loaded 39,321 compounds
   ✓ Columns: ['pert_id', 'cmap_name', 'target', 'moa', 'canonical_smiles', 'inchi_key', 'compound_aliases']
   ✓ Using 'canonical_smiles' for molecular structures
   ✓ Compounds with valid SMILES: 33,531

🧪 APPLYING ALL CHEMICAL FILTERS

⚠️  Filtering only 13,823 compounds present in training data
   Compounds to check: 18,187
Initial compounds: 18,187

CHEMICAL FILTER 1: Molecular Weight (60-1000 Da)


[19:50:56] SMILES Parse Error: syntax error while parsing: restricted
[19:50:56] SMILES Parse Error: check for mistakes around position 1:
[19:50:56] restricted
[19:50:56] ^
[19:50:56] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:50:56] SMILES Parse Error: syntax error while parsing: restricted
[19:50:56] SMILES Parse Error: check for mistakes around position 1:
[19:50:56] restricted
[19:50:56] ^
[19:50:56] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:50:56] SMILES Parse Error: syntax error while parsing: restricted
[19:50:56] SMILES Parse Error: check for mistakes around position 1:
[19:50:56] restricted
[19:50:56] ^
[19:50:56] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:50:58] SMILES Parse Error: syntax error while parsing: restricted
[19:50:58] SMILES Parse Error: check for mistakes around position 1:
[19:50:58] restricted
[19:50:58] ^
[19:50:58] SMILES Parse Error: Fai

  ✓ Passed: 13,429 compounds
  ✗ Failed (out of MW range): 66 compounds
  ⚠️  No SMILES: 331 compounds
  ⚠️  Invalid SMILES: 16 compounds

  MW distribution of failed compounds:
    • Mean: 1400.6 Da
    • Min: 59.1 Da
    • Max: 2430.9 Da

CHEMICAL FILTER 2: Covalent Motifs (≤1)
Checking 20 covalent SMARTS patterns...
  ✓ Successfully compiled 20 SMARTS patterns


[19:50:59] SMILES Parse Error: syntax error while parsing: restricted
[19:50:59] SMILES Parse Error: check for mistakes around position 1:
[19:50:59] restricted
[19:50:59] ^
[19:50:59] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:50:59] SMILES Parse Error: syntax error while parsing: restricted
[19:50:59] SMILES Parse Error: check for mistakes around position 1:
[19:50:59] restricted
[19:50:59] ^
[19:50:59] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:50:59] SMILES Parse Error: syntax error while parsing: restricted
[19:50:59] SMILES Parse Error: check for mistakes around position 1:
[19:50:59] restricted
[19:50:59] ^
[19:50:59] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:51:02] SMILES Parse Error: syntax error while parsing: restricted
[19:51:02] SMILES Parse Error: check for mistakes around position 1:
[19:51:02] restricted
[19:51:02] ^
[19:51:02] SMILES Parse Error: Fai

  ✓ Passed: 13,131 compounds
  ✗ Failed (>1 motifs): 596 compounds

  Distribution of failed compounds:
    • Mean motifs: 2.2
    • Max motifs: 4

CHEMICAL FILTER 3: NIBR Structure Flags (≤9)
Using official NIBR filter catalog (Schuffenhauer et al. 2020)
  ✓ Catalog contains 444 filter patterns


[19:51:04] SMILES Parse Error: syntax error while parsing: restricted
[19:51:04] SMILES Parse Error: check for mistakes around position 1:
[19:51:04] restricted
[19:51:04] ^
[19:51:04] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:51:04] SMILES Parse Error: syntax error while parsing: restricted
[19:51:04] SMILES Parse Error: check for mistakes around position 1:
[19:51:04] restricted
[19:51:04] ^
[19:51:04] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:51:06] SMILES Parse Error: syntax error while parsing: restricted
[19:51:06] SMILES Parse Error: check for mistakes around position 1:
[19:51:06] restricted
[19:51:06] ^
[19:51:06] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:51:38] SMILES Parse Error: syntax error while parsing: restricted
[19:51:38] SMILES Parse Error: check for mistakes around position 1:
[19:51:38] restricted
[19:51:38] ^
[19:51:38] SMILES Parse Error: Fai

  ✓ Passed: 12,380 compounds
  ✗ Failed (>9 flags): 1,944 compounds

  Flag distribution (all compounds):
    • Mean flags: 2.32
    • Median flags: 0
    • Max flags: 10

  Flag count histogram:
    • 0 flags: 5,822 compounds
    • 1 flags: 1,048 compounds
    • 2 flags: 111 compounds
    • 3 flags: 12 compounds
    • 10 flags: 1,944 compounds

  Failed compounds statistics:
    • Mean flags: 10.00
    • Max flags: 10

CHEMICAL FILTER 4: BRENK Criteria


[19:52:05] SMILES Parse Error: syntax error while parsing: restricted
[19:52:05] SMILES Parse Error: check for mistakes around position 1:
[19:52:05] restricted
[19:52:05] ^
[19:52:05] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:52:05] SMILES Parse Error: syntax error while parsing: restricted
[19:52:05] SMILES Parse Error: check for mistakes around position 1:
[19:52:05] restricted
[19:52:05] ^
[19:52:05] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:52:05] SMILES Parse Error: syntax error while parsing: restricted
[19:52:05] SMILES Parse Error: check for mistakes around position 1:
[19:52:05] restricted
[19:52:05] ^
[19:52:05] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[19:52:11] SMILES Parse Error: syntax error while parsing: restricted
[19:52:11] SMILES Parse Error: check for mistakes around position 1:
[19:52:11] restricted
[19:52:11] ^
[19:52:11] SMILES Parse Error: Fai

  ✓ Passed: 9,843 compounds
  ✗ Failed (BRENK violations): 5,386 compounds

📊 CHEMICAL FILTER SUMMARY
  Initial compounds: 18,187
  Passed Filter 1 (MW): 13,429
  Passed Filter 2 (Covalent): 13,131
  Passed Filter 3 (NIBR): 12,380
  Passed Filter 4 (BRENK): 9,843
  ────────────────────────────────────────────────────────────────────────────────
  ✅ Passed ALL filters: 9,434
  ❌ Removed: 8,753
  Retention rate: 51.9%

📊 FILTERING TRAINING DATA
  Compounds in training data: 13,823
  Compounds passed filters: 9,434
  Compounds to remove: 4,389
  Removing 276,536 samples from 4,389 compounds...

✅ FILTERING COMPLETE
  Before chemical filters:
    • Samples: 583,491
    • Compounds: 13,823
  After chemical filters:
    • Samples: 306,955 (52.6%)
    • Compounds: 9,434 (68.2%)
    • Cell lines: 57
  Removed:
    • Samples: 276,536
    • Compounds: 4,389

📊 Comparison with paper (SI page 2):
  Paper: 425,242 obs, 9,597 compounds, 52 cells
  Ours:  306,955 obs, 9,434 compounds, 57 cells
  Matc

In [3]:
import pandas as pd

data_processed = pd.read_pickle("E:/科研/Models/drugreflector/processed_data/training_data_lincs2020_final.pkl")

data_processed['folds']

array([1, 0, 2, ..., 1, 0, 2])